In [1]:
import sys, csv, json
import requests
from newsapi.articles import Articles
from newsapi.sources import Sources
import numpy as np
import csv, json
import pandas as pd
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import unicodedata
import math
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops

C:\Users\Shireen\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
key = '01d1817afecf4671b81608bdb5a7b988'

In [3]:
a = Articles(API_KEY=key)
s = Sources(API_KEY=key)

In [4]:
class APIKeyException(Exception):
    def __init__(self, message): 
        self.message = message 

class InvalidQueryException(Exception):
    def __init__(self, message): 
        self.message = message

In [5]:
class ArchiveAPI(object):
    def __init__(self, key=None):
        self.key = key
        self.root = 'http://api.nytimes.com/svc/archive/v1/{}/{}.json?api-key={}' 
        if not self.key:
            nyt_dev_page = 'http://developer.nytimes.com/docs/reference/keys'
            exception_str = 'Warning: API Key required. Please visit {}'
            raise NoAPIKeyException(exception_str.format(nyt_dev_page))

    def query(self, year=None, month=None, key=None,):
        """
        Calls the archive API and returns the results as a dictionary.
        :param key: Defaults to the API key used to initialize the ArchiveAPI class.
        """
        if not key: 
            key = self.key
            
        if (year < 1882) or not (0 < month < 13):
            # currently the Archive API only supports year >= 1882
            exception_str = 'Invalid query: See http://developer.nytimes.com/archive_api.json'
            raise InvalidQueryException(exception_str)
            
        url = self.root.format(year, month, key)
        r = requests.get(url)
        return r.json()


api = ArchiveAPI('8a8ce3a995aa4dfca47e09b29d4c5256')


years = [2018]
months = [5]

for year in years:
    for month in months:
        mydict = api.query(year, month)
        #file_str = '/Users/user/Desktop/stock_rnn_data/' + str(year) + '-' + '{:02}'.format(month) + '.json'
        file_str = 'C:/Users/Shireen/Testdata/' + str(year) + '-' + '{:02}'.format(month) + '.json'
        with open(file_str, 'w') as fout:
            try:
                json.dump(mydict, fout)
            except:
                pass
        fout.close()

In [7]:
#18th march - djia ONLY
with open('C:/Users/Shireen/Testdata/13DJIA.csv', 'r',encoding="utf-8") as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    # Converting the csv file reader to a lists 
    data_list = list(spamreader)
    data_list = np.asarray(data_list)
    print(data_list)

[['Date' 'Open' 'High' ... 'Close' 'Adj Close' 'Volume']
 ['2007-12-31' '13364.160156' '13364.730469' ... '13264.820313'
  '13264.820313' '167240000']
 ['2008-01-02' '13261.820313' '13279.540039' ... '13043.959961'
  '13043.959961' '239580000']
 ...
 ['2018-05-09' '24399.179688' '24586.480469' ... '24542.539063'
  '24542.539063' '361580000']
 ['2018-05-10' '24591.660156' '24794.990234' ... '24739.529297'
  '24739.529297' '304210000']
 ['2018-05-11' '24758.640625' '24868.650391' ... '24831.169922'
  '24831.169922' '274150000']]


In [8]:
header = data_list[0] 
data_list = data_list[1:] 
data_list = np.asarray(data_list)
print(data_list)

[['2007-12-31' '13364.160156' '13364.730469' ... '13264.820313'
  '13264.820313' '167240000']
 ['2008-01-02' '13261.820313' '13279.540039' ... '13043.959961'
  '13043.959961' '239580000']
 ['2008-01-03' '13044.120117' '13137.929688' ... '13056.719727'
  '13056.719727' '200620000']
 ...
 ['2018-05-09' '24399.179688' '24586.480469' ... '24542.539063'
  '24542.539063' '361580000']
 ['2018-05-10' '24591.660156' '24794.990234' ... '24739.529297'
  '24739.529297' '304210000']
 ['2018-05-11' '24758.640625' '24868.650391' ... '24831.169922'
  '24831.169922' '274150000']]


In [9]:
selected_data = data_list[:, [0, 4, 6]]

In [10]:
df = pd.DataFrame(data=selected_data[0:,1:],
             index=selected_data[0:,0],
                                columns=['close', 'adj close'],
                                        dtype='float64')

In [11]:
print (df.tail())

                   close    adj close
2018-05-07  24357.320313  307670000.0
2018-05-08  24360.210938  344940000.0
2018-05-09  24542.539063  361580000.0
2018-05-10  24739.529297  304210000.0
2018-05-11  24831.169922  274150000.0


In [12]:
df1 = df
idx = pd.date_range('01-01-2008', '05-13-2018')
df1.index = pd.DatetimeIndex(df1.index)
df1 = df1.reindex(idx, fill_value=np.NaN)
# df1.count() # gives 2518 count
interpolated_df = df1.interpolate() # Fill in the gap
interpolated_df.count() # gives 3651 count

close        3785
adj close    3785
dtype: int64

In [13]:
print (df1.tail(10))

                   close    adj close
2018-05-04  24262.509766  329480000.0
2018-05-05           NaN          NaN
2018-05-06           NaN          NaN
2018-05-07  24357.320313  307670000.0
2018-05-08  24360.210938  344940000.0
2018-05-09  24542.539063  361580000.0
2018-05-10  24739.529297  304210000.0
2018-05-11  24831.169922  274150000.0
2018-05-12           NaN          NaN
2018-05-13           NaN          NaN


In [14]:
interpolated_df = interpolated_df[3:]
print (interpolated_df.tail(10))

                   close    adj close
2018-05-04  24262.509766  329480000.0
2018-05-05  24294.113282  322210000.0
2018-05-06  24325.716797  314940000.0
2018-05-07  24357.320313  307670000.0
2018-05-08  24360.210938  344940000.0
2018-05-09  24542.539063  361580000.0
2018-05-10  24739.529297  304210000.0
2018-05-11  24831.169922  274150000.0
2018-05-12  24831.169922  274150000.0
2018-05-13  24831.169922  274150000.0


In [1]:
#============#

In [15]:
date_format = ["%Y-%m-%dT%H:%M:%SZ", "%Y-%m-%dT%H:%M:%S+%f"]

def try_parsing_date(text):
    for fmt in date_format:
        try:
            return datetime.strptime(text, fmt).strftime('%Y-%m-%d')
        except ValueError:
            pass
    raise ValueError('no valid date format found')

In [16]:
years = [2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008]
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
dict_keys = ['pub_date', 'headline'] #, 'lead_paragraph']
articles_dict = dict.fromkeys(dict_keys)

In [17]:
# Filtering list for type_of_material
type_of_material_list = ['blog', 'brief', 'news', 'editorial', 'op-ed', 'list','analysis']
# Filtering list for section_name
section_name_list = ['business', 'national', 'world', 'u.s.' , 'politics', 'opinion', 'tech', 'science',  'health']
news_desk_list = ['business', 'national', 'world', 'u.s.' , 'politics', 'opinion', 'tech', 'science',  'health', 'foreign']

In [18]:
current_date = '2018-05-13'
from datetime import datetime

current_article_str = ''

In [19]:
interpolated_df["articles"] = ''
count_articles_filtered = 0
count_total_articles = 0
count_main_not_exist = 0               
count_unicode_error = 0     
count_attribute_error = 0

In [20]:
for year in years:  # search for every month
    for month in months:
        file_str = 'C:/Users/Shireen/Testdata/' + str(year) + '-' + '{:02}'.format(month) + '.json'
        with open(file_str) as data_file:    
            NYTimes_data = json.load(data_file)
        count_total_articles = count_total_articles + len(NYTimes_data["response"]["docs"][:]) #add article number 
        for i in range(len(NYTimes_data["response"]["docs"][:])): # search in every docs for type of material or section = in the list
            try:
                if any(substring in NYTimes_data["response"]["docs"][:][i]['type_of_material'].lower() for substring in type_of_material_list):
                    if any(substring in NYTimes_data["response"]["docs"][:][i]['section_name'].lower() for substring in section_name_list):
                        #count += 1
                        count_articles_filtered += 1
                        #print 'i: ' + str(i) dick_key = ['pub_date', 'headline']
                        articles_dict = { your_key: NYTimes_data["response"]["docs"][:][i][your_key] for your_key in dict_keys }
                        articles_dict['headline'] = articles_dict['headline']['main'] # Selecting just 'main' from headline
                        #articles_dict['headline'] = articles_dict['lead_paragraph'] # Selecting lead_paragraph
                        date = try_parsing_date(articles_dict['pub_date'])
                        #print 'article_dict: ' + articles_dict['headline']
                        # putting same day article str into one str
                        if date == current_date:
                            current_article_str = current_article_str + '. ' + articles_dict['headline']
                        else:  
                            interpolated_df.set_value(current_date, 'articles', interpolated_df.loc[current_date, 'articles'] + '. ' + current_article_str)
                            current_date = date
                            #interpolated_df.set_value(date, 'articles', current_article_str)
                            #print str(date) + current_article_str
                            current_article_str = articles_dict['headline']
                        # For last condition in a year
                        if (date == current_date) and (i == len(NYTimes_data["response"]["docs"][:]) - 1): 
                            interpolated_df.set_value(date, 'articles', current_article_str)   
                        
             #Exception for section_name or type_of_material absent
            except AttributeError:
                #print 'attribute error'
                #print NYTimes_data["response"]["docs"][:][i]
                count_attribute_error += 1
                # If article matches news_desk_list if none section_name found
                try:
                    if any(substring in NYTimes_data["response"]["docs"][:][i]['news_desk'].lower() for substring in news_desk_list):
                            #count += 1
                            count_articles_filtered += 1
                            #print 'i: ' + str(i)
                            articles_dict = { your_key: NYTimes_data["response"]["docs"][:][i][your_key] for your_key in dict_keys }
                            articles_dict['headline'] = articles_dict['headline']['main'] # Selecting just 'main' from headline
                            #articles_dict['headline'] = articles_dict['lead_paragraph'] # Selecting lead_paragraph
                            date = try_parsing_date(articles_dict['pub_date'])
                            #print 'article_dict: ' + articles_dict['headline']
                            if date == current_date:
                                current_article_str = current_article_str + '. ' + articles_dict['headline']
                            else:  
                                interpolated_df.set_value(current_date, 'articles', interpolated_df.loc[current_date, 'articles'] + '. ' + current_article_str)
                                current_date = date
                                #interpolated_df.set_value(date, 'articles', current_article_str)
                                #print str(date) + current_article_str
                                current_article_str = articles_dict['headline']
                            # For last condition in a year
                            if (date == current_date) and (i == len(NYTimes_data["response"]["docs"][:]) - 1): 
                                interpolated_df.set_value(date, 'articles', current_article_str)   
                
                except AttributeError:
                    pass
                pass
            except KeyError:
                print ('key error')
                #print NYTimes_data["response"]["docs"][:][i]
                count_main_not_exist += 1
                pass   
            except TypeError:
                print ("type error")
                #print NYTimes_data["response"]["docs"][:][i]
                count_main_not_exist += 1
                pass


key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error


key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error


key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error


key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error


key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error


...

key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error


key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error


key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error


key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error


key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error
key error


In [21]:
print (count_articles_filtered)
print (count_total_articles)                     
print (count_main_not_exist)
print (count_unicode_error)

426297
1166814
64707
0


In [22]:
## Putting all articles if no section_name or news_desk not found
for date, row in interpolated_df.T.iteritems():   
    if len(interpolated_df.loc[date, 'articles']) <= 400:
        #print interpolated_df.loc[date, 'articles']
        #print date
        month = date.month
        year = date.year
        file_str = 'C:/Users/Shireen/Testdata/' + str(year) + '-' + '{:02}'.format(month) + '.json'
        with open(file_str) as data_file:    
            NYTimes_data = json.load(data_file)
        count_total_articles = count_total_articles + len(NYTimes_data["response"]["docs"][:])
        interpolated_df.set_value(date.strftime('%Y-%m-%d'), 'articles', '')
        for i in range(len(NYTimes_data["response"]["docs"][:])):
            try:
                
                articles_dict = { your_key: NYTimes_data["response"]["docs"][:][i][your_key] for your_key in dict_keys }
                articles_dict['headline'] = articles_dict['headline']['main'] # Selecting just 'main' from headline
                #articles_dict['headline'] = articles_dict['lead_paragraph'] # Selecting lead_paragraph       
                pub_date = try_parsing_date(articles_dict['pub_date'])
                #print 'article_dict: ' + articles_dict['headline']
                if date.strftime('%Y-%m-%d') == pub_date: 
                    interpolated_df.set_value(pub_date, 'articles', interpolated_df.loc[pub_date, 'articles'] + '. ' + articles_dict['headline'])  
                
            except KeyError:
                print ('key error')
                #print NYTimes_data["response"]["docs"][:][i]
                #count_main_not_exist += 1
                pass   
            except TypeError:
                print ("type error")
                #print NYTimes_data["response"]["docs"][:][i]
                #count_main_not_exist += 1
                pass

type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error

In [23]:
# Saving the data as pickle file
#interpolated_df.to_pickle('/Users/user/Desktop/stock_rnn_data/pickled_ten_year_filtered_lead_para.pkl')
interpolated_df.to_pickle('C:/Users/Shireen/Testdata/14pickled_data.pkl')


# Save pandas frame in csv form
#interpolated_df.to_csv('/Users/user/Desktop/stock_rnn_data/sample_interpolated_df_10_years_filtered_lead_para.csv',sep='\t', encoding='utf-8')
interpolated_df.to_csv('C:/Users/Shireen/Testdata/14sample_interpolated_df_10_years_filtered_lead_para.csv', sep='\t', encoding='utf-8')



# Reading the data as pickle file
dataframe_read = pd.read_pickle('C:/Users/Shireen/Testdata/14pickled_data.pkl')

In [24]:
df_stocks = pd.read_pickle('C:/Users/Shireen/Testdata/14pickled_data.pkl')
print (df_stocks.tail())

                   close    adj close  \
2018-05-09  24542.539063  361580000.0   
2018-05-10  24739.529297  304210000.0   
2018-05-11  24831.169922  274150000.0   
2018-05-12  24831.169922  274150000.0   
2018-05-13  24831.169922  274150000.0   

                                                     articles  
2018-05-09  . What’s Kim Jong-un’s Surname? Mike Pompeo Is...  
2018-05-10  . 24 Art Exhibitions to View in N.Y.C. This We...  
2018-05-11  . Nine Teaching Ideas for Using Music to Inspi...  
2018-05-12  . Here Are the Biggest Stories in American Pol...  
2018-05-13                                                     


In [25]:
df_stocks['prices'] = df_stocks['adj close']

In [26]:
# selecting the prices and articles
df_stocks = df_stocks[['prices', 'articles']]
print (df_stocks.tail())

                 prices                                           articles
2018-05-09  361580000.0  . What’s Kim Jong-un’s Surname? Mike Pompeo Is...
2018-05-10  304210000.0  . 24 Art Exhibitions to View in N.Y.C. This We...
2018-05-11  274150000.0  . Nine Teaching Ideas for Using Music to Inspi...
2018-05-12  274150000.0  . Here Are the Biggest Stories in American Pol...
2018-05-13  274150000.0                                                   


In [27]:
df_stocks['articles'] = df_stocks['articles'].map(lambda x: x.lstrip('.-'))

In [28]:
df_stocks.tail()

,prices,articles
2018-05-09,361580000.0,What’s Kim Jong-un’s Surname? Mike Pompeo Is ...
2018-05-10,304210000.0,24 Art Exhibitions to View in N.Y.C. This Wee...
2018-05-11,274150000.0,Nine Teaching Ideas for Using Music to Inspir...
2018-05-12,274150000.0,Here Are the Biggest Stories in American Poli...
2018-05-13,274150000.0,


In [29]:
df_stocks.head()

,prices,articles
2008-01-04,304210000.0,Brooklyn Blaze Kills a Fire Lieutenant. Boats...
2008-01-05,305040000.0,New Hampshire and Those Labor Ads. A Cold Bla...
2008-01-06,305870000.0,"Weekend in Melbourne, Australia. Back in Time..."
2008-01-07,306700000.0,How Riders Responded to M.T.A.’s ‘Say Somethi...
2008-01-08,322690000.0,"Temperatures in the Mid-60s, and Warm to the ..."


In [30]:
df = df_stocks[['prices']].copy()
df.head()
df["compound"] = ''
df["neg"] = ''
df["neu"] = ''
df["pos"] = ''
df.head()

,prices,compound,neg,neu,pos
2008-01-04,304210000.0,,,,
2008-01-05,305040000.0,,,,
2008-01-06,305870000.0,,,,
2008-01-07,306700000.0,,,,
2008-01-08,322690000.0,,,,


In [31]:
df_stocks.T

,2008-01-04 00:00:00,2008-01-05 00:00:00,2008-01-06 00:00:00,2008-01-07 00:00:00,2008-01-08 00:00:00,2008-01-09 00:00:00,2008-01-10 00:00:00,2008-01-11 00:00:00,2008-01-12 00:00:00,2008-01-13 00:00:00,...,2018-05-04 00:00:00,2018-05-05 00:00:00,2018-05-06 00:00:00,2018-05-07 00:00:00,2018-05-08 00:00:00,2018-05-09 00:00:00,2018-05-10 00:00:00,2018-05-11 00:00:00,2018-05-12 00:00:00,2018-05-13 00:00:00
prices,3.0421e+08,3.0504e+08,3.0587e+08,3.067e+08,3.2269e+08,3.329e+08,3.2533e+08,3.0189e+08,2.8305e+08,2.6421e+08,...,3.2948e+08,3.2221e+08,3.1494e+08,3.0767e+08,3.4494e+08,3.6158e+08,3.0421e+08,2.7415e+08,2.7415e+08,2.7415e+08
articles,Brooklyn Blaze Kills a Fire Lieutenant. Boats...,New Hampshire and Those Labor Ads. A Cold Bla...,"Weekend in Melbourne, Australia. Back in Time...",How Riders Responded to M.T.A.’s ‘Say Somethi...,"Temperatures in the Mid-60s, and Warm to the ...",What Makes Wah-High So Special. A Landmarks F...,"Get Into It, It’s Mad Easy. Remembering the O...",Podcast: A Rare Pivotal Primary in New York. ...,"On the Trail. As Snow Falls, Packers Advance....",The Elusive Affordable Family Ski Weekend. We...,...,How Israelis See the World. Teenagers in The ...,What’s on TV Saturday: The Rock & Roll Hall o...,"What’s on TV Sunday: ‘Dateline,’ With Meek Mi...",Yankees’ Gleyber Torres Upstages Himself With...,Review: When the ‘Light Shining’ on Revolutio...,What’s Kim Jong-un’s Surname? Mike Pompeo Is ...,24 Art Exhibitions to View in N.Y.C. This Wee...,Nine Teaching Ideas for Using Music to Inspir...,Here Are the Biggest Stories in American Poli...,


In [32]:
sid = SentimentIntensityAnalyzer()
for date, row in df_stocks.T.iteritems():
    try:
        sentence = unicodedata.normalize('NFKD', df_stocks.loc[date, 'articles'])
        ss = sid.polarity_scores(sentence)
        df.set_value(date, 'compound', ss['compound'])
        df.set_value(date, 'neg', ss['neg'])
        df.set_value(date, 'neu', ss['neu'])
        df.set_value(date, 'pos', ss['pos'])
    except TypeError:
        print (df_stocks.loc[date, 'articles'])
        print (date)

In [33]:
df.tail()

,prices,compound,neg,neu,pos
2018-04-04,388540000.0,-0.9966,0.121,0.798,0.08
2018-04-05,347790000.0,-0.9993,0.142,0.789,0.069
2018-04-06,395985226.0,-0.998,0.123,0.804,0.072
2018-04-07,395985226.0,-0.9977,0.182,0.722,0.096
2018-04-08,395985226.0,-0.4019,0.064,0.875,0.061


In [34]:
df.head()

,prices,compound,neg,neu,pos
2008-01-04,304210000.0,-0.9993,0.122,0.808,0.07
2008-01-05,305040000.0,-0.9921,0.115,0.81,0.075
2008-01-06,305870000.0,0.9989,0.097,0.773,0.13
2008-01-07,306700000.0,-0.991,0.093,0.833,0.074
2008-01-08,322690000.0,-0.9972,0.119,0.783,0.098


In [35]:
datasetNorm = (df - df.mean()) / (df.max() - df.min())
datasetNorm.reset_index(inplace=True)
del datasetNorm['index']
datasetNorm['next_prices'] = datasetNorm['prices'].shift(-1)
datasetNorm.tail(10)

,prices,compound,neg,neu,pos,next_prices
3738,0.349214,0.168273,-0.0981416,0.0160168,0.0767261,0.363398
3739,0.363398,0.397708,-0.497422,-0.849985,-0.418644,0.377583
3740,0.377583,-0.101467,0.128477,-0.0105613,-0.103829,0.391767
3741,0.391767,-0.092616,-0.123321,0.0425949,-0.0019776,0.249908
3742,0.249908,-0.0981668,-0.108933,0.0437023,-0.0251257,0.243053
3743,0.243053,-0.100667,-0.0621703,0.0337355,-0.0482739,0.193086
3744,0.193086,-0.102017,0.0133692,0.0237687,-0.0991998,0.252183
3745,0.252183,-0.101367,-0.0549761,0.04038,-0.0853109,0.252183
3746,0.252183,-0.101217,0.157254,-0.0504284,0.0258002,0.252183
3747,0.252183,0.196727,-0.267206,0.119007,-0.136237,NaN
